### Maximum likelihood estimates for Dirichlet parameters
- We parameterize the distribution by its mean $\vec{m}$ and its sharpness $\alpha$. (Sometimes this is written $\vec{u} = \vec{m}\alpha$.)
- Here we consider maximum likelihood estimators under the assumption $\alpha m_i >> 1$.
- Let $J$ denote the number of classes (e.g. $J=2$ for the Beta).

$$\begin{align}
[x | \vec{m}, \alpha] &= \frac{\Gamma (\alpha)}{\prod_j \Gamma(\alpha m_j)} \prod_j x_j^{\alpha m_j - 1} \\
                      &= \frac{\Gamma (\alpha)}{\Gamma (\alpha - (J - 1))}  \binom{\alpha - J}{\alpha m_1 - 1, \alpha m_2 - 1, ...}\prod_j x_j^{\alpha m_j - 1} \\
\end{align}$$

For large $\alpha$ we may approximate the multinomial coefficient in terms of the entropy. We define a new distribution $\vec{n}$ closely related to the mean by $n_j = (\alpha m_j - 1) / (\alpha - J)$ which represents the presence of each term in the multinomial coefficient. We may then approximate the multinomial coefficient as $\exp \left\{ - (\alpha - J) H(\vec{n}) \right\}$, where $H(\vec{n})$ is the entropy of $\vec{n}$.

After approximating the first term by $\alpha ^ {J-1}$ and taking logarithms we get,

$$\begin{align}
\log [x | \vec{m}, \alpha] &\approx (J - 1) \log \alpha - (\alpha - J) H(\vec{n}) + \sum_i(\alpha m_j - 1)\log x_j \\
                           &\approx (J - 1) \log \alpha - (\alpha - J) H(\vec{n}) + (\alpha - J) \sum_j n_j\log x_j \\
                           &\approx (J - 1) \log \alpha - (\alpha - J) D_{KL}(\vec{n}, \vec{x_j})
\end{align}$$

where $D_{KL}$ is the relative entropy. For large $\alpha$ we make the approximation $\vec{n} = \vec{m}$. Which gives the maximum likelihood estimation procedure a natural interpretaion: we choose $\hat{m}_{ML}$ to minimize the average divergence between $\vec{m}$ and the data. The MLE for $\vec{m}$ does not depend on $\alpha$. We will return to this problem in a moment. Once we know $\vec{m}$ we have,

$$ \frac{d}{d \log \alpha}\log [x | \vec{m}, \alpha] \approx (J - 1) - (\alpha - J) D_{KL}(\vec{n}, \vec{x_j}) $$

which yields,

$$\hat{\alpha}_{ML} \approx \frac{J-1}{D_{KL}}$$

When we have multiple $x^{(i)}$ in the sample it can be shown that the same formula obtains using the mean divergence. Likewise finding $\hat{m}$ requires minimizing the mean divergence over the sample. We consider this now. We want to minimize,
$$\left< D_{KL} (\vec{m}, \vec{x}) \right>$$
which is equal to
$$\sum_j m_j (\log m_j - \left< \log x_j \right> )$$

We are subject to the constraint $\sum_j m_j - 1 = 0$, so using the method of Lagrange multipliers we get,

$$\begin{align}
\log m_j - \left< \log x_j \right> + 1 + \lambda &= 0 \\
\log m_j &= \left< \log x_j \right> - (\lambda + 1) \\
\log m_j &\propto \exp \left< \log x_j \right> \\
\end{align}$$

The constant of proportionality ($\exp -(\lambda + 1)$) is the same for all $m_j$, so to find $\hat{m}$ we simply take the geometric mean of the sample and normalize it to have a sum of 1.




## Numerical Dirichlet Parameter Estimation

We want to maximize the log likelihood:
$$f(\bar{x} | \bar{\alpha}) = \log \Gamma(\alpha) - \sum_i \log \Gamma(\alpha_i) + \sum_i (\alpha_i - 1) \left< \log x_i \right>$$
where $\alpha = \sum_i \alpha_i$. To maximize the log likelihood we want $\nabla f = \mathbf{0}$ where
$$\frac{\partial f}{\partial \alpha_i} = \psi(\alpha) - \psi(\alpha_i) + \left< \log x_i \right>$$
where $\psi$ is the digamma function. We employ Newton's method $\mathbf{H} (\mathbf{d\alpha}) = -\nabla f$. The Hessian is given by
$$\frac{\partial^2 f}{\partial \alpha_i \partial \alpha_j} = \psi_1(\alpha) - \psi_1(\alpha_i) \delta_{ij}$$
where $\psi_1$ is the derivative of the digamma function (the so-called trigamma function). We apply $ (\mathbf{d\alpha}) = - \mathbf{H^{-1}} \nabla f$ and iterate.

If we want to get reasonable results for small samples it will be helpful to implement weight decay by putting a broad exponential prior over the $\alpha_i$. If we assign a prior of $e^{-\alpha_i/\lambda}$ then the log posterior changes by $-\alpha_i/\lambda$ and the gradient decreases by $-1/\lambda$.





## Method

1. We need to reconstruct the prior based on 80 samples from the prior. (Properly this is an inference problem: we define a prior over possible priors and update on the samples.) In practice we model the prior as a mixture of Dirichlet distributions and do MLE.
4. Get the satellite data $y$
5. Update to get $z^{+}$ (and $w^{+} = 1 - z^{+}$). Use an optimal transport strategy (inverse CDF, CDF).
6. Update to get $k_{max}^{+}$ conditioned on $w$. Do optimal transport.
7. Look up the parameters for the appropriate Dirichlet from step (1). Take a new posterior sample from this distribution. Optimal transport is accomplished by doing invCDF o CDF one coordinate at a time.

### Marginal distribution for conditioned Dirichlet

We are interested in the marginal / conditional probability $x_j | \{ x_k \}$. Let $\{ x_k \}$ denote the remaining variables over which we are marginalizing. Then,
$$[x_i | \{x_j\}] \propto x_j^{u_j - 1}(1-\sum_i x_i - x_j)^{(\sum_k u_k) - 1}$$.

So $x_j$ is distributed as a Beta within the remaining space. Precisely, $(x_j / (1-\sum_i x_i))$ is distributed as $Beta(u_j, \sum_k u_k)$. This means we can easily find the CDF and invCDF. The idea will be to transport ourselves one coordinate at a time.


We want to find $[ k^- | w^- ]$ so that we can transform $k^-$ back into a uniform.
1. Find the likelihood of each class $[ w^- | k^- ]$
2. Multiply by the prior class probabilities (i.e. mixing factors in the prior)
3. Normalize

We want to transform our current class to a uniform random variable
1. Stack up the probabilities for $[ k^- | w^- ]$ in the established order.
2. Look at the interval, e.g. (0.03, 0.08), in which our class falls.
3. Pick uniformly from that interval.

We want to use this uniform random variable to choose our sample from the posterior $[k^+ | w^+]$
1. Stack up the probabilities $[ k^+ | w^+ ]$ in the established order.
2. Take our uniform RV and check what interval it lies in. This tells us our class. (Think of arithmetic coding.)


